In [1]:
from gasp3.gt.prop.feat import get_ext

In [2]:
PATH_SHP = '/home/jasp/tejo_pnt2.shp'
PATH__JS = '/home/jasp/tejo_pnt.js'

In [3]:
# Get Extent

left, right, bottom, top = get_ext(PATH_SHP)

In [4]:
from osgeo import ogr
import time
import datetime

dtSrc = ogr.GetDriverByName('ESRI Shapefile').Open(PATH_SHP)

lyr = dtSrc.GetLayer()

In [5]:
COORDINATES = []
UNIX_TIMES  = []

for feat in lyr:
    __ID = feat.GetField('ID')
    geom = feat.GetGeometryRef()
    __XX = geom.GetX()
    __YY = geom.GetY()
    __TT = feat.GetField('HORA')
    
    hora, __min, seg = __TT.split(':')
    dt = datetime.datetime(2019, 9, 5, int(hora), int(__min), int(seg))
    
    UNIX_TIMES.append(str(int(time.mktime(dt.timetuple()) * 1000)))
    
    COORDINATES.append([__XX, __YY])

In [6]:
JS_CODE = """
var tejo = {{
    "type" : "Feature",
    "geometry" : {{
        "type": "MultiPoint",
        "coordinates" : [{}]
    }},
    "properties" : {{
        "title" : "tejo",
        "path_options" : {{ "color" : "red" }},
        "time" : [{}]
    }},
    "bbox" : [
        [{}, {}],
        [{}, {}],
        [{}, {}],
        [{}, {}]
    ]
}};

var demoTracks = [tejo];
""".format(
    ",".join(["[{}, {}]".format(str(coord[0]), str(coord[1])) for coord in COORDINATES]),
    ",".join(UNIX_TIMES),
    str(left), str(bottom), str(left), str(top),
    str(right), str(top), str(right), str(bottom)
).strip()

with open(PATH__JS, 'a') as f:
    f.write(JS_CODE)